In [1]:
import random
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import torchvision.transforms as transforms
from model import cifar_mobilenet, cifar_discriminator_model, cifar_decoder, cifar_pseudo, vgg16
from model1 import BottomModelForCifar10, TopModelForCifar10
from vfl import VFLNN, Client, Server
import torch.backends.cudnn as cudnn



In [4]:
a = torch.tensor([1, 2, 3], dtype=torch.float32)
b = torch.tensor([4, 5, 6], dtype=torch.float32)
loss = F.mse_loss(a,b)
print(loss)

tensor(9.)


sss

In [2]:
use_cuda = torch.cuda.is_available()
if use_cuda:
    id = 'cuda:'+ '0'
    device = torch.device(id)
    torch.cuda.set_device(id)
    # cudnn.benchmark = True
else:
    device = torch.device('cpu')

print(device)

cuda:0


In [3]:
torch.manual_seed(3407)
random.seed(3407)
np.random.seed(3407)
cudnn.deterministic = True
cudnn.benchmark = False

In [4]:
def split_data(data):
    x_a = data[:, :, :, 0:16]
    x_b = data[:, :, :, 16:32]
    return x_a, x_b

In [5]:
import copy


cinic_transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)),
            ])


train_dataset = torchvision.datasets.CIFAR10(root='./data', train = True, transform=cinic_transform, download=True)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train = False, transform=cinic_transform, download=True)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers = 4, pin_memory = True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers = 4, pin_memory = True)

target_bottom1,target_top = cifar_mobilenet(level=2)
target_bottom2 = copy.deepcopy(target_bottom1)

# 模型放到GPU上
target_bottom1, target_bottom2, target_top = target_bottom1.to(device), target_bottom2.to(device), target_top.to(device)
target_client1 = Client(target_bottom1)
target_client2 = Client(target_bottom2)
target_server = Server(target_top)

Files already downloaded and verified
Files already downloaded and verified


In [6]:
target_iterator = iter(train_dataloader)
target_data, target_labels = next(target_iterator)
target_data, target_labels = target_data.to(device), target_labels.to(device)

In [26]:
x_a, x_b = split_data(target_data) # torch.Size([64, 3, 32, 16])
# print(x_a.shape)
# target_bottom1_intermediate = target_client1.forward(x_a)
# target_bottom2_intermediate = target_client2.forward(x_b)
# inter1 = target_bottom1_intermediate.detach()
# print(inter1.shape)  # torch.Size([64, 64, 16, 8])  ---> 合并为[64, 64, 16, 16]

# pseudo_model, _ = vgg16(level=2, batch_norm = True)
# pseudo_model = pseudo_model.to(device)
# pseudo_model_ouput = pseudo_model(x_a)

# x_a和x_b的shape
# torch.Size([64, 3, 32, 16])
# torch.Size([64, 3, 32, 16])

# print(pseudo_model_ouput.shape)
test_out_put1 = target_bottom1(x_a)
test_out_put2 = target_bottom2(x_b)
discriminator_input_shape = test_out_put1.shape[1:]
print(x_a.shape)
# decoder
decoder = cifar_decoder(discriminator_input_shape, 2, 3)
decoder = decoder.to(device)
x_a_recvoer = decoder(torch.cat((test_out_put1, test_out_put2), 3))
print(x_a_recvoer.shape)

# 鉴别器
# discriminator = cifar_discriminator_model(discriminator_input_shape, 2)
# discriminator = discriminator.to(device)
# adv_target_logits = discriminator(test_out_put)
# print(adv_target_logits.shape)

torch.Size([64, 3, 32, 16])
[DECODER] activation:  None
torch.Size([64, 3, 32, 32])


In [7]:
print(target_bottom1(x_a).shape)

torch.Size([64, 64, 16, 8])


In [ ]:
# Convert the tensor to a numpy array and transpose the dimensions


# fig, axes = plt.subplots(1, 10, figsize=(20, 2))
# plt.subplots_adjust(wspace=0, hspace=-.05)
# for i in range(10):
#     image = target_data[i].cpu().numpy().transpose((1, 2, 0))
#     image = image * 0.5 + 0.5 # Unnormalize the image
#     axes[i].imshow(image)
#     axes[i].axis('off')

# plt.show()

NameError: name 'plt' is not defined

In [9]:
dataset_shape = train_dataset[0][0].shape
test_data = torch.ones(2,dataset_shape[0], dataset_shape[1], dataset_shape[2]).to(device)
aa = target_bottom1(test_data)
bb = target_top(aa)
print(aa.shape)
print(bb.shape)
print(bb)

torch.Size([2, 64, 16, 16])
torch.Size([2, 10])
tensor([[-0.0262,  0.0297,  0.0209, -0.0080,  0.0303,  0.0173, -0.0062, -0.0299,
         -0.0277,  0.0249],
        [-0.0262,  0.0297,  0.0209, -0.0080,  0.0303,  0.0173, -0.0062, -0.0299,
         -0.0277,  0.0249]], device='cuda:0', grad_fn=<AddmmBackward0>)


In [ ]:
client1_optimizer = optim.Adam(target_bottom1.parameters(), lr=0.01)
client2_optimizer = optim.Adam(target_bottom2.parameters(), lr=0.01)
server_optimizer = optim.Adam(target_top.parameters(), lr=0.01)
client_optimizer = [client1_optimizer, client2_optimizer]
target_vflNN = VFLNN(target_client1, target_client2, target_server, client_optimizer, server_optimizer)

# test_out_put = target_vflNN.client2(x_a)
# print(test_out_put.shape)


for i in range(30):
    print("Epoch: ", i) 
    target_vflNN.train()
    train_loss = 0
    for batchidx, (data, target) in enumerate(train_dataloader):
        
        data, target_label = data.to(device), target.to(device)
        target_vflNN.zero_grads()
        x_a, x_b = split_data(data)

        target_vflNN_output = target_vflNN(x_a, x_b)
        # 计算loss
        target_vflNN_loss = F.cross_entropy(target_vflNN_output, target_label)
        
        # 反向传播
        target_vflNN_loss.backward()
        # 整体vflNN的反向传播
        target_vflNN.backward()

        train_loss += target_vflNN_loss.item() * data.size(0)
        # 更新模型
        target_vflNN.step()
    train_loss = train_loss / len(train_dataloader.dataset)
    print("======loss=======")
    print(train_loss)
        
    target_vflNN.eval()
    
    print("---------------------------testtesttest---------------------------")
    correct = 0
    total = 0
    with torch.no_grad():
        for test_data, test_target in test_dataloader:
            test_data, test_target = test_data.to(device), test_target.to(device)
            x_a, x_b = split_data(test_data)
            outputs = target_vflNN(x_a, x_b)
            _, predicted = torch.max(outputs.data, 1)
            total += test_target.size(0)
            correct += (predicted == test_target).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy of the network on the test images: {accuracy:.2f}%')


torch.Size([64, 64, 16, 8])
